In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RAG 基本實作
<table>
<td style="text-align: center">
  <a href="https://colab.research.google.com/github/TWCkaijin/GDGC-Gemini-bootcamp/blob/main/RAG/Vertex%20AI%20RAG%20Search%20Workshop.ipynb">
    <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
  </a>
</td>        
<td style="text-align: center">
  <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FTWCkaijin%2FGDGC-Gemini-bootcamp%2Frefs%2Fheads%2Fmain%2FRAG%2FVertex%20AI%20RAG%20Search%20Workshop.ipynb">
    <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
  </a>
</td>

</table>

| | |
|-|-|
|原作者 | [Lavi Nigam](https://github.com/lavinigam-gcp) |

DIY version of RAG [**building_DIY_multimodal_qa_system_with_mRAG.ipynb**](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/qa-ops/building_DIY_multimodal_qa_system_with_mRAG.ipynb)

## 概述
檢索增強生成（Retrieval Augmented Generation, RAG）已成為讓大型語言模型（LLMs）存取外部數據的重要範式，同時也作為一種機制，能有效減少幻覺現象（hallucinations）。

在此筆記本中，您將學習如何進行多模態 RAG，透過結合文字與圖片的財務文件進行問答。

## Gemini
- Gemini 是 Google DeepMind 開發的一系列生成式 AI 模型，專為多模態應用設計。
- Gemini API 提供對 Gemini 3.0 Pro 、Gemini 2.5 Pro 和 Gemini 2.5 Flash 模型的存取。

## 比較文字型與多模態 RAG
多模態 RAG 相較於文字型 RAG 具有以下幾項優勢：

1. 增強的知識存取能力：
多模態 RAG 能處理文字與視覺資訊，為大型語言模型提供更豐富、更全面的知識基礎。

2. 改進的推理能力：
藉由整合視覺線索，多模態 RAG 能在不同數據模態之間進行更明智的推斷。

此筆記本將教您如何在 Vertex AI 中，結合 Gemini API、文字嵌入 和 多模態嵌入，構建文件搜尋引擎。

透過實際操作範例，您將學會如何為文件來源建構一個多媒體豐富的中繼資料庫，實現跨多元資訊流的搜尋、比較與推理功能。

## 單元目標
本筆記本逐步提供建構文件搜尋引擎的指南，透過多模態檢索增強生成（RAG）實現以下功能：

- 提取並儲存包含文字的文件中繼資料(metadata)，並為文件生成嵌入向量
- 使用文字查詢搜尋中繼資料(metadata)，並向LLM提供檢索結果並回答問題

## 工具:
請注意，專案必須要啟動以下GCP API服務:
- Vertex AI API

## 基本設定

### 下載安裝 Vertex AI SDK for Python 及其他相依套件

In [ ]:
%pip install --upgrade --user google-cloud-aiplatform pymupdf rich colorama

### 重啟以確保套件成功載入

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### 驗證Colab 環境(僅Colab需要)

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### 初始化 GCP 專案設定
請注意，專案必須要啟動以下API服務:
- Vertex AI API

https://console.cloud.google.com</br>

以下輸入您的專案ID以及欲使用的伺服器區域。

In [ ]:
import sys

PROJECT_ID = "[YOUR-PROJECT-ID]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# if not running on Colab, try to get the PROJECT_ID automatically
if "google.colab" not in sys.modules:
    import subprocess

    PROJECT_ID = subprocess.check_output(
        ["gcloud", "config", "get-value", "project"], text=True
    ).strip()

print(f"Your project ID is: {PROJECT_ID}")

### 初始化 Vertex AI

In [ ]:
import sys

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### 導入函式庫

In [ ]:
from IPython.display import Markdown, display
from rich.markdown import Markdown as rich_Markdown
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image

### 定義及載入 Gemini 2.5 Pro 和 Gemini 2.5 Flash 模型

In [ ]:
text_model = GenerativeModel("gemini-2.5-flash")
multimodal_model = GenerativeModel("gemini-2.5-flash")

### 下載示範用檔案以及額外套件

For original code: (`intro_multimodal_rag_utils.py`) directly on [GitHub](https://storage.googleapis.com/github-repo/rag/intro_multimodal_rag/intro_multimodal_rag_old_version/intro_multimodal_rag_utils.py).

In [ ]:
# download documents and images used in this notebook
!gsutil -m rsync -r gs://github-repo/rag/intro_multimodal_rag/intro_multimodal_rag_old_version .
!mkdir pdf
!curl -o ./pdf/example.pdf https://raw.githubusercontent.com/TWCkaijin/GDGC-Gemini-bootcamp/main/RAG/Korvin%20Fantasy.pdf
print("Download completed")

## 1. 建立包含文字metadata

### 範例資料內容

在此筆記本中，我們將使用的原始數據為一篇「虛構角色的生平故事」

單一、獨立且無出現在網路上的內容，適合進行多模態檢索增強生成（RAG）的實驗與學習。


### 匯入輔助函數以建立metadata
在構建多模態 RAG 系統之前，需先準備文件中所有文字與圖片的metadata。為了便於引用與參考，metadata應包含關鍵元素，例如頁碼、檔案名稱等。

接下來，您將從這些metadata中生成嵌入向量，這些嵌入向量是執行相似性搜尋時的必要條件。

In [ ]:
from intro_multimodal_rag_utils import get_document_metadata

### 從文件中提取並儲存文字和圖片的metadata

我們剛剛匯入了一個名為 ```get_document_metadata()``` 的函數。這個函數會從文件中提取文字和圖片的中繼資料，並返回兩個資料框，分別為 ```text_metadata``` 和 ```image_metadata``` 。如果你想了解更多有關 ```get_document_metadata()``` 函數如何使用 Gemini 和嵌入模型實現的細節，你可以直接查看[source code](https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/retrieval-augmented-generation/utils/intro_multimodal_rag_utils.py)。

提取並儲存文字和圖片metadata的原因在於，僅使用其中一個資料類型不足以得出相關的答案。例如，相關答案可能以視覺形式存在於文件中，但文字型 RAG 無法考慮到視覺圖像。你稍後會在這本筆記本中探索這個範例。








在下一步，我們將使用這個函數來提取並儲存文件中文字和圖片的metadata。請注意，以下的程式區塊可能需要幾分鐘才能完成執行。

注意事項：

目前的實現最適用於以下情況：

* 文件中包含文字和圖片的組合。
* 文件中的表格以圖片形式呈現。
* 文件中的圖片不需要太多上下文信息。</br>

另外，
* 你也可以使用常規的 RAG 方法。可以參考這份檔案 [RAG_text_only](https://github.com/kevin6449/LANGCHAIN_RAG/blob/main/LangChain_RAG.ipynb)
* 如果文件包含額外特定領域的知識，可以將這些信息傳遞至下方的提示語中。

<div class="alert alert-block alert-warning">
<b>⚠️ 不要傳送超過50頁的資料，你可能會遇到流量限制 ⚠️</b></br>
⚠️ 如果你遇到了其他形式的流量限制，請開啟下方的add_sleep_after_page並設定sleep_time_after_page ⚠️
</div>

In [ ]:
pdf_folder_path = "pdf/"


image_description_prompt = """
Explain what is going on in the image.
please describe the whole plot according to the image
"""

# Extract text and image metadata from the PDF document
text_metadata_df, image_metadata_df = get_document_metadata(
    multimodal_model, 
    pdf_folder_path,
    image_save_dir="images",
    image_description_prompt=image_description_prompt,
    embedding_size=1408,
    add_sleep_after_page = True,
    sleep_time_after_page = 11, 
)

print("\n\n --- Completed processing. ---")

#### 檢視處理過的文字metadata
以下的程式區塊將產生一個metadata表格，描述不同部分的文字metadata，包括：

- text: 來自頁面的原始文字
- text_embedding_page: 頁面上原始文字的嵌入向量
- chunk_text: 將原始文字分割成較小的區塊
- chunk_number: 每個文字區塊的索引
- text_embedding_chunk: 每個文字區塊的嵌入向量







In [ ]:
text_metadata_df.head()

### 匯入輔助函數以實現 RAG
你將匯入以下函數，這些函數將在本筆記本的其餘部分中用來實現 RAG：

- ```get_similar_text_from_query()```：根據文字查詢，使用餘弦相似度算法從文件中找出相關的文字。此函數使用中繼資料中的文字嵌入向量來計算結果，並可以根據最高分數、頁碼/區塊號或嵌入向量大小來過濾結果。
- ```print_text_to_text_citation()```：印出從 ```get_similar_text_from_query()``` 函數檢索到的文字來源（引用）和細節。
- ```get_similar_image_from_query()```：根據圖片路徑或圖片，從文件中找出相關的圖片。此函數使用中繼資料中的圖片嵌入向量。
- ```print_text_to_image_citation()```：印出從 ```get_similar_image_from_query()``` 函數檢索到的圖片來源（引用）和細節。
- ```get_gemini_response()```：與 Gemini 模型互動，基於文字和圖片輸入的結合來回答問題。
- ```display_images()```：顯示一系列圖片，這些圖片可以是路徑或 PIL 圖片對象。

In [ ]:
from intro_multimodal_rag_utils import (
    display_images,
    get_gemini_response,
    get_similar_image_from_query,
    get_similar_text_from_query,
    print_text_to_image_citation,
    print_text_to_text_citation,
)

## 2. 文字搜尋
讓我們從一個簡單的問題開始，看看使用文字嵌入向量的簡單文字搜尋是否能夠回答這個問題。

In [ ]:
query = "what is this file mainly about"

### 使用文字搜尋來尋找相關資料

In [ ]:
# Matching user text query with "chunk_embedding" to find relevant chunks.
matching_results_text = get_similar_text_from_query(
    query,
    text_metadata_df,
    column_name="text_embedding_chunk",
    top_n=7,
    chunk_text=True,
)

# Print the matched text citations
print_text_to_text_citation(matching_results_text, print_top=False, chunk_text=True)

我們可以發現，第一個高分結果包含了我們需要的信息，但仔細檢查後發現，它提到該信息存在於“以下”表格中。該表格數據是以圖片形式存在，而非文字，因此除非能處理圖片及其數據，否則很可能會錯過這些信息。

不過，讓我們將相關的文字區塊輸入 Gemini 1.0 Pro 模型，看看它是否能在考慮文件中所有區塊的情況下給出我們想要的答案。這就像是基本的文字型 RAG 實現。

In [ ]:

print("\n **** Result: ***** \n")

# All relevant text chunk found across documents based on user query
context = "\n".join(
    [value["chunk_text"] for key, value in matching_results_text.items()]
)

instruction = f"""Answer the question with the given context.
If the information is not available in the context, return "not available in the context" and explain the situation.
Question: {query}
Context: {context}
Answer:
"""

model_input = instruction

# Generate Gemini response with streaming output
Markdown(get_gemini_response(
    text_model, 
    model_input=model_input,
    stream=True,
    generation_config=GenerationConfig(temperature=0.2),
))

#### 接下來，讓我們來問問更多關於主角的故事吧~~

In [ ]:
while(True):
  query = input()

  # Matching user text query with "chunk_embedding" to find relevant chunks.
  matching_results_text = get_similar_text_from_query(
      query,
      text_metadata_df,
      column_name="text_embedding_chunk",
      top_n=7,
      chunk_text=True,
  )

  # Print the matched text citations
  #print_text_to_text_citation(matching_results_text, print_top=False, chunk_text=True)


  print("\n **** Result: ***** \n")

  # All relevant text chunk found across documents based on user query
  context = "\n".join(
      [value["chunk_text"] for key, value in matching_results_text.items()]
  )

  instruction = f"""Answer the question with the given context.
  If the information is not available in the context, return "not available in the context" and explain the situation.
  Question: {query}
  Context: {context}
  Answer:
  """

  # Prepare the model input
  model_input = instruction

  # Generate Gemini response with streaming output
  print(get_gemini_response(
      text_model,  # we are passing Gemini 1.0 Pro
      model_input=model_input,
      stream=True,
      generation_config=GenerationConfig(temperature=0.2),
  ))


## 3. 使用文字來做圖像搜索

我們一樣先建立一個Prompt

In [ ]:
query = "How is the final battle looks?"

接下來，使用已經寫好的工具函式`get_similar_image_from_query`，他會回傳相近的圖片以及圖片的描述資料。

In [ ]:
matching_results_image = get_similar_image_from_query(
    text_metadata_df,
    image_metadata_df,
    query=query,
    column_name="text_embedding_from_image_description",  # Use image description text embedding
    image_emb=False,  # Use text embedding instead of image embedding
    top_n=3,
    embedding_size=1408,
)

print("\n **** Result: ***** \n")

# Display the top matching image
display(matching_results_image[0]["image_object"])

接下來，建立整合的Prompt，並傳送給Gemini

In [ ]:
print("\n **** Result: ***** \n")

# All relevant text chunk found across documents based on user query
context = f"""Image: {matching_results_image[0]['image_object']}
Description: {matching_results_image[0]['image_description']}
"""

instruction = f"""Answer the question in JSON format with the given context of Image and its Description. Only include value.
Question: {query}
Context: {context}
Answer:
"""

model_input = instruction

Markdown(
    get_gemini_response(
        multimodal_model_flash, 
        model_input=model_input,
        stream=True,
        generation_config=GenerationConfig(temperature=1),
    )
)

使用`print_text_to_image_citation`可以取得資料的出處

In [ ]:
Markdown(print_text_to_image_citation(matching_results_image, print_top=True))

使用圖片來查詢圖片

In [ ]:
# You can find a similar image as per the images you have in the metadata.
# In this case, you have a table (picked from the same document source) and you would like to find similar tables in the document.
image_query_path = "tac_table_revenue.png"

# Print a message indicating the input image
print("***Input image from user:***")

# Display the input image
Image.load_from_file(image_query_path)

接下來使用Image embedder 來嵌入圖片。

In [ ]:
# Search for Similar Images Based on Input Image and Image Embedding

matching_results_image = get_similar_image_from_query(
    text_metadata_df,
    image_metadata_df,
    query=query,  # Use query text for additional filtering (optional)
    column_name="mm_embedding_from_img_only",  # Use image embedding for similarity calculation
    image_emb=True,
    image_query_path=image_query_path,  # Use input image for similarity calculation
    top_n=3,  # Retrieve top 3 matching images
    embedding_size=1408,  # Use embedding size of 1408
)

print("\n **** Result: ***** \n")

# Display the Top Matching Image
display(
    matching_results_image[0]["image_object"]
)  # Display the top matching image object (Pillow Image)

印出圖片出處

In [ ]:
# Display citation details for the top matching image
print_text_to_image_citation(
    matching_results_image, print_top=True
)  # Print citation details for the top matching image

列印出結果

In [ ]:
# Check Other Matched Images (Optional)
# You can access the other two matched images using:

print("---------------Matched Images------------------\n")
display_images(
    [
        matching_results_image[0]["img_path"],
        matching_results_image[1]["img_path"],
    ],
    resize_ratio=0.5,
)